# Visualização interativa do ENEM 2023

#### Links dos dados utilizados:
- ENEM: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem
- Referência geoespacial dos municípios brasileiros: https://github.com/kelvins/municipios-brasileiros/blob/main/csv/municipios.csv

In [1]:
# Bibliotecas utilizadas
import pandas as pd

In [2]:
# Carregando os dados do ENEM
enem_data = pd.read_csv('MICRODADOS_ENEM_2023.csv', encoding='latin-1', sep=';')

In [3]:
# Mostrando o header
enem_data.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A


In [4]:
# Contagem de participantes por cidade (código do IBGE da cidade: participantes)
value_counts_per_city_total = enem_data.CO_MUNICIPIO_PROVA.value_counts()
print(value_counts_per_city_total)

3550308    157220
3304557    116997
2304400     74112
5300108     72975
2927408     69514
            ...  
1200328        66
1302108        65
1400159        51
1200435        41
2605459        34
Name: CO_MUNICIPIO_PROVA, Length: 1750, dtype: int64


In [5]:
# Filtrando os dados do enem para somente os que tiveram presença em todas as provas
enem_data_present = enem_data[
    (enem_data.TP_PRESENCA_CN == 1) & 
    (enem_data.TP_PRESENCA_CH == 1) & 
    (enem_data.TP_PRESENCA_LC == 1) &
    (enem_data.TP_PRESENCA_MT == 1)
]

# Contagem de participantes presentes por cidade (código do IBGE da cidade: participantes)
value_counts_per_city_present = enem_data_present.CO_MUNICIPIO_PROVA.value_counts()
print(value_counts_per_city_present)

3550308    105986
3304557     74830
2304400     50375
5300108     50267
2927408     43937
            ...  
5102793        39
1400159        27
1302108        25
2605459        24
1200435        16
Name: CO_MUNICIPIO_PROVA, Length: 1750, dtype: int64


In [6]:
# Alinhando as pandas.Series por código de cidade do IBGE
aligned_present_total = value_counts_per_city_present.align(value_counts_per_city_total, join='right', fill_value=0)

# Proporção de participantes presentes por cidade (código do IBGE da cidade: porcentagem de participantes)
percentage_present_per_city = (aligned_present_total[0] / aligned_present_total[1]) * 100
print(round(percentage_present_per_city, 2))

3550308    67.41
3304557    63.96
2304400    67.97
5300108    68.88
2927408    63.21
           ...  
1200328    62.12
1302108    38.46
1400159    52.94
1200435    39.02
2605459    70.59
Name: CO_MUNICIPIO_PROVA, Length: 1750, dtype: float64


In [7]:
# Carregando os dados geoespaciais dos municípios do Brasil
geo_data = pd.read_csv('municipios.csv')

In [8]:
# Mostrando o header
geo_data.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [9]:
# Convertendo value_counts_per_city_total de um pandas.Series para um pandas.DataFrame
value_counts_per_city_total = value_counts_per_city_total.reset_index()
value_counts_per_city_total.columns = ['codigo_ibge', 'total_applicants']

# Convertendo percentage_present_per_city de um pandas.Series para um pandas.DataFrame
percentage_present_per_city = percentage_present_per_city.reset_index()
percentage_present_per_city.columns = ['codigo_ibge', 'percentage_present']

# Colocando CO_MUNICIPIO_PROVA:codigo_ibge como referência para a união dos dataframes
value_counts_per_city_total = value_counts_per_city_total.rename(columns={'CO_MUNICIPIO_PROVA': 'codigo_ibge'})
percentage_present_per_city = percentage_present_per_city.rename(columns={'CO_MUNICIPIO_PROVA': 'codigo_ibge'})

# Mesclando o dataframe de dados geoespaciais com a respectiva porcentagem de participação no ENEM
geo_plus_education_data = pd.merge(geo_data, value_counts_per_city_total, on='codigo_ibge', how='left', suffixes=('', '_education'))
geo_plus_education_data = pd.merge(geo_plus_education_data, percentage_present_per_city, on='codigo_ibge', how='left', suffixes=('', '_percentage'))
print(geo_plus_education_data.head())

   codigo_ibge                 nome  latitude  longitude  capital  codigo_uf  \
0      5200050      Abadia de Goiás -16.75730   -49.4412        0         52   
1      3100104  Abadia dos Dourados -18.48310   -47.3916        0         31   
2      5200100            Abadiânia -16.19700   -48.7057        0         52   
3      3100203               Abaeté -19.15510   -45.4444        0         31   
4      1500107           Abaetetuba  -1.72183   -48.8788        0         15   

   siafi_id  ddd       fuso_horario  total_applicants  percentage_present  
0      1050   62  America/Sao_Paulo               NaN                 NaN  
1      4001   34  America/Sao_Paulo               NaN                 NaN  
2      9201   62  America/Sao_Paulo               NaN                 NaN  
3      4003   37  America/Sao_Paulo             658.0           71.580547  
4       401   91  America/Sao_Paulo            6178.0           67.983166  


In [10]:
# Exportando o dataframe para um arquivo .csv
geo_plus_education_data.to_csv('geo_plus_education_data.csv', index=False)